# 1. Initializations

## 1.1 General imports

In [ ]:
### general
import datetime
import pickle

### data management
import pandas as pd
import numpy as np

### machine learning (scikit-learn)
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Lasso
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import root_mean_squared_error
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_regression, mutual_info_regression, RFE, RFECV
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

### graphical
import matplotlib.pyplot as plt
# for jupyter notebook management
%matplotlib inline
import seaborn as sns


## 1.2 General dataframe functions

In [ ]:
import smartcheck.dataframe_common as dfc

## 1.3 Specific preprocessing classes

In [ ]:
import smartcheck.preprocessing_project_specific as pps

# 2. Loading and preprocessing

In [ ]:
df_cpt_raw = dfc.load_dataset_from_config('velo_comptage_ml_ready_data', sep=',', index_col=0)

if df_cpt_raw is not None and isinstance(df_cpt_raw, pd.DataFrame):
    df_cpt = df_cpt_raw.copy()

In [ ]:
df_cpt.info()

## 2.1 Column preprocessing pipelines

In [ ]:
no_change_cols = [
    "identifiant_du_compteur",
    "comptage_horaire",
    "orientation_compteur",
    "latitude",
    "longitude",
    "arrondissement",
    "temperature_2m_c",
    "rain_mm",
    "snowfall_cm",
    "elevation",
]

preproc_transf = ColumnTransformer(
    transformers=[
        (
            "filter",
            pps.ColumnFilterTransformer(columns_to_keep=no_change_cols),
            no_change_cols
        ),
        (
            "datetime",
            pps.DatetimePreprocessingTransformer(timestamp_col="date_et_heure_de_comptage"),
            ["date_et_heure_de_comptage"]
        ),
        (
            "meteo_code",
            pps.MeteoCodePreprocessingTransformer(code_col="weather_code_wmo_code"),
            ["weather_code_wmo_code"]
        ),
    ],
    remainder="drop",
)

preproc_transf.set_output(transform="pandas")

df = pd.DataFrame(preproc_transf.fit_transform(df_cpt)) # type: ignore
df = df.rename(columns={
    col: col.split("__")[-1] for col in df.columns if "__" in col
})

In [ ]:
df.head()
display(df.select_dtypes(include=np.number).describe())
display(df.select_dtypes(include='object').describe())
display(df.select_dtypes(include=np.number).corr())

## 2.2 Time series processing and analysis

### With Resample (only numeric variables) and seasonal decompose

In [ ]:
col_temp = ['date_et_heure_de_comptage_local']
col_keep = ['comptage_horaire']
df_res = df[col_temp+col_keep].set_index(col_temp)
# resample sur l'index
df_res = df_res.resample('h').sum()
display(df_res.info())
display(df_res.head())

In [ ]:
decomp_res = seasonal_decompose(df_res[:24*30], model='additive', period=24)
fig = decomp_res.plot()
fig.set_figwidth(12)
fig.set_figheight(12)
plt.show()
# Ce graphique permet de préestimer d, D, et s dans le modèle SARIMA

### With GroupBy and Periods (categorical and numerical variables) and ACF/PACF decompose

In [ ]:
# definition des variables utiles (temporelle/numeriques/categorielles)
col_temp = ['date_et_heure_de_comptage_local']
num_col_keep = list(df.select_dtypes(include=np.number).columns)
cat_col_keep = [
    'weather_code_wmo_code_category',
    'orientation_compteur',
    'arrondissement'
]
# extraction des variables pertinentes et reindexation via la variable temporelle
df_gpd = df[col_temp+num_col_keep+cat_col_keep].copy().set_index(col_temp)
# création d'une colonne périodique 
df_gpd['hour_period'] = df_gpd.index.to_period('h') # type: ignore
# cumuls pour les variables numériques par période
df_grby_cum = df_gpd.groupby('hour_period')[num_col_keep].sum()
# proportions pour les variables categorielles par période
for cat in cat_col_keep:
    cat_grby_cnt = df_gpd.groupby(['hour_period']+[cat]).size().unstack(fill_value=0)
    cat_grby_props = cat_grby_cnt.div(cat_grby_cnt.sum(axis=1), axis=0)
    df_grby_cum = df_grby_cum.join(cat_grby_props)
df_gpd = df_grby_cum.copy()
display(df_grby_cum.info())
display(df_grby_cum.describe())

#### Matrice de correlation des variables numériques

In [ ]:
plt.figure(figsize=(20, 20))  # Taille de la figure
corr_mat = df_grby_cum.corr()
sns.heatmap(corr_mat, annot=True, fmt=".2f", cmap="coolwarm", mask=np.triu(corr_mat))
plt.show()

In [ ]:
# Affichage des autocorrelations successives avec visualisation du résidu
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, figsize=(15,25))
# NB : la série est déjà stationnaire donc pas de différentiation simple nécessaire
y = df_gpd['comptage_horaire'][:3000]
# différenciation saisonnière 1 (saisonalité sur 24h intuitée)
s1 = 24
# différenciation saisonnière 2 (sous saisonalité sur 48h intuitée)
s2 = 24*7

y.plot(ax=ax1)
ax1.set_title('Données brutes')
pd.plotting.autocorrelation_plot(y, ax=ax2)
ax2.set_title(f'Autocorrelation données brutes')

# 1ère différenciation
y_s1 = y.diff(s1).dropna()
y_s1.plot(ax=ax3)
ax3.set_title(f'Données résidus après 1ère Différenciation {s1}h')
pd.plotting.autocorrelation_plot(y_s1, ax=ax4)
ax4.set_title(f'Autocorrelation après 1ère Différenciation {s1}h')

# 2ème différenciation
y_s2 = y_s1.diff(s2).dropna()
y_s2.plot(ax=ax5)
ax5.set_title(f'Données résidus après 2ème Différenciation {s2}h')
pd.plotting.autocorrelation_plot(y_s2, ax=ax6)
ax6.set_title(f'Autocorrelation après 2ème Différenciation {s2}h')

plt.show()

In [ ]:
# test statistique de la stationnarité
result = adfuller(y)
print("Stationnarité sans différentiation",result[1]) 
result  = adfuller(y_s1)
print("Stationnarité après 1ère différenciation",result[1]) 
result = adfuller(y_s2)
print("Stationnarité après 2nde différenciation",result[1]) 

# 3. Feature engineering

In [ ]:
# Features / target separation
features = df_gpd.drop(columns=['comptage_horaire'])
target = df_gpd['comptage_horaire']
test_ratio = 0.10

In [ ]:
# train / test separation 
index_ratio = int(df_gpd.shape[0]*test_ratio//1)
X_train = features.iloc[:-index_ratio].copy()
X_test = features.iloc[-index_ratio:].copy()
y_train = target.iloc[:-index_ratio].copy()
y_test = target.iloc[-index_ratio:].copy()

display(X_train.shape, y_train.shape)
display(X_test.shape, y_test.shape)

In [ ]:
# standardisation des échelles
# Preprocessing des variables explicatives d'entrainement et de test (scaler)
s_scaler = StandardScaler().fit(X_train)
# Recuperation des propriétés du dataframe perdues (nparray) avec le scaler
X_train = pd.DataFrame(s_scaler.fit_transform(X_train), index=X_train.index)
X_test = pd.DataFrame(s_scaler.transform(X_test), index=X_test.index)

## 3.1 Feature selection

### 3.1.1 KBest

In [ ]:
fe_skb_mir = SelectKBest(score_func = mutual_info_regression, k=13)
fe_skb_mir.fit(X_train, y_train)
mask = fe_skb_mir.get_support()
plt.matshow(mask.reshape(1,-1), cmap = 'gray_r')
plt.xticks(ticks=range(X_train.shape[1]), labels=X_train.columns, rotation=90)
plt.yticks([])
plt.xlabel('Axe des features')
plt.show()

In [ ]:
fe_skb_fr = SelectKBest(score_func = f_regression, k=13)
fe_skb_fr.fit(X_train, y_train)
mask = fe_skb_fr.get_support()
plt.matshow(mask.reshape(1,-1), cmap = 'gray_r')
plt.xticks(ticks=range(X_train.shape[1]), labels=X_train.columns, rotation=90)
plt.yticks([])
plt.xlabel('Axe des features')
plt.show()

### 3.1.2 RFE

In [ ]:
clf_lr = LinearRegression()
fe_rfe = RFE(estimator=clf_lr, step=1, n_features_to_select = 13)
fe_rfe.fit(X_train, y_train)
mask = fe_rfe.get_support()
plt.matshow(mask.reshape(1,-1), cmap = 'gray_r')
plt.xticks(ticks=range(X_train.shape[1]), labels=X_train.columns, rotation=90)
plt.yticks([])
plt.xlabel('Axe des features')
plt.show()

In [ ]:
ranking = fe_rfe.ranking_
print(ranking)
plt.matshow(ranking.reshape(1,-1), cmap = 'gray')
plt.xlabel('Axe des features')
plt.xticks(ticks=range(X_train.shape[1]), labels=X_train.columns, rotation=90)
plt.yticks([])
plt.show()

### 3.1.3 RFECV

In [ ]:
cv_kf = TimeSeriesSplit(n_splits=5)
clf_lr = LinearRegression()
fe_rfecv = RFECV(estimator=clf_lr, cv = cv_kf, step=1)
fe_rfecv.fit(X_train, y_train)
mask = fe_rfecv.get_support()
plt.matshow(mask.reshape(1,-1), cmap = 'gray_r')
plt.xticks(ticks=range(X_train.shape[1]), labels=X_train.columns, rotation=90)
plt.yticks([])
plt.xlabel('Axe des features')
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))
for i in range(5):
    ax1.plot(fe_rfecv.cv_results_[f'split{i}_test_score'])
ax1.set_xlabel('Nombre de features')
ax1.set_ylabel('Score')
ax1.set_title('Score par fold de test pour chaque itération')
ax2.plot(fe_rfecv.cv_results_['mean_test_score'])
ax2.set_xlabel('Nombre de features')
ax2.set_ylabel('Score')
ax2.set_title('Score moyen en cross validation')
plt.show()
print("Nombre de features retenues :", fe_rfecv.n_features_)

### 3.1.4 LASSO + GridSearch

In [ ]:
cv_kf = TimeSeriesSplit(n_splits=5)
clf_lasso = Lasso(alpha = 1, max_iter=5000)
alpha_grid = {'alpha':[1/i for i in range(1,10)]}
search_gscv = GridSearchCV(estimator = clf_lasso, param_grid = alpha_grid, cv=cv_kf, scoring = 'neg_mean_squared_error')
search_gscv.fit(X_train, y_train)
best_clf_lasso = search_gscv.best_estimator_
print("Meilleur alpha :", search_gscv.best_params_)
fe_sfm = SelectFromModel(estimator=best_clf_lasso, threshold=1e-10, prefit=True)
mask = fe_sfm.get_support()
plt.matshow(mask.reshape(1,-1), cmap = 'gray_r')
plt.xticks(ticks=range(X_train.shape[1]), labels=X_train.columns, rotation=90)
plt.yticks([])
plt.xlabel('Axe des features')
plt.show()

## 3.2 Dimension Reduction

### 3.2.1 PCA

#### Training

In [ ]:
# récuperation des coordonnées de primary components par défaut
fe_pca = PCA()
coord_pca_train = fe_pca.fit_transform(X_train)

#### interpretation

In [ ]:
# Graphique de la part de variance expliquée par composante 
fig_pca, (ax1_pca, ax2_pca) = plt.subplots(2, 1, figsize=(8,6))
ax1_pca.set_xlim(0, 30)
ax1_pca.set_xticks(range(30))
ax1_pca.set_xlabel('Nombre de composantes')
ax1_pca.set_ylabel('Part de variance expliquée')
ax1_pca.plot(fe_pca.explained_variance_ratio_)
# Graphique de la part de variance expliquée cumulée avec seuil de 90%
ax2_pca.set_xlim(0, 30)
ax2_pca.set_xticks(range(30))
ax2_pca.set_xlabel('Nombre de composantes')
ax2_pca.set_ylabel('Variance expliquée cumulée')
ax2_pca.plot(fe_pca.explained_variance_ratio_.cumsum())
ax2_pca.axhline(y=0.9, color='r', linestyle='--')
plt.tight_layout()
plt.show()
# Graphique de la part de variance des X composantes principales par rapport au reste des composantes
x = 6
pca_var_ratios = list(fe_pca.explained_variance_ratio_[:x])
pca_var_ratios.append(sum(fe_pca.explained_variance_ratio_[x:]))

plt.pie(
    pca_var_ratios, 
    labels=[f'PC{i}' for i in range(x)]+['Autres PCx'], 
    autopct='%1.2f%%'
)
plt.show()

#### Feature projection

In [ ]:
# projection sur le premier axe de la composante 0
component_0 = fe_pca.components_[0,:]
explained_var_0 = fe_pca.explained_variance_[0]
corr_axe0 = component_0 * np.sqrt(explained_var_0)
display(corr_axe0.shape)

# projection sur le second axe de la composante 1
component_1 = fe_pca.components_[1,:]
explained_var_1 = fe_pca.explained_variance_[1]
corr_axe1 = component_1 * np.sqrt(explained_var_1)
display(corr_axe1.shape)

# creation d'un dataset avec les deux axes
charges_factorielles = pd.DataFrame(
    [corr_axe0, corr_axe1],
    columns=features.columns,
    index=['Axe 0', 'Axe 1'])
display(charges_factorielles)

def draw_correlation_circle(df_charges_factorielles, pca, arrow_length=0.01, label_rotation=0):
    fig, ax = plt.subplots(figsize=(15, 15))
    for i, var in enumerate(df_charges_factorielles.columns):
        x = df_charges_factorielles.loc['Axe 0', var]
        y = df_charges_factorielles.loc['Axe 1', var]
        ax.arrow(0, 0, 
                 x*(1-arrow_length), 
                 y*(1-arrow_length), 
                 head_width=arrow_length, 
                 head_length=arrow_length, alpha=0.5, fc='blue', ec='blue')
        ax.text(x, y, var,
                ha='center', va='center',
                fontsize=9, rotation=label_rotation, clip_on=True)
    circle = plt.Circle((0, 0), 1, facecolor='none', edgecolor="#1EE4C9")
    ax.add_artist(circle)
    ax.set_xlim(-1.1, 1.1)
    ax.set_ylim(-1.1, 1.1)
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlabel('Axe 0 (PC0)')
    ax.set_ylabel('Axe 1 (PC1)')
    ax.set_title('Cercle des Corrélations')
    plt.grid()
    plt.show()
    
# Tracer le cercle de corrélation
draw_correlation_circle(charges_factorielles, fe_pca, arrow_length=0.05)

#### Features data (coordinates) projection and interpretation with target

In [ ]:
display(coord_pca_train.shape)

df_plot = pd.DataFrame(coord_pca_train[:150, :2], columns=["PC0", "PC1"])  
df_plot["target"] = pd.Series(y_train[:150]).reset_index(drop=True)

# Affichage du graphique de projection des coordonnées sur les deux axes principaux (seaborn)
plt.figure(figsize=(8,6))
sns.scatterplot(
    data=df_plot,
    x='PC0', y='PC1',
    hue='target',           # coloration selon la variable cible
    palette='coolwarm',         # ou 'viridis', 'coolwarm', etc.
    s=30, edgecolor='k'
)
plt.title("Projection PCA - composantes 0 et 1 pour 150 échantillons")
plt.xlabel("PC0")
plt.ylabel("PC1")
plt.legend(title='Cible')
plt.grid(True)
plt.tight_layout()
plt.show()

# Affichage du graphique de projection des coordonnées sur les deux axes principaux (matplotlib)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(df_plot.PC0, df_plot.PC1,  c = df_plot.target, cmap=plt.cm.coolwarm, alpha = .7, s = 30)
ax.set_title("Données projetées sur les 2 composantes")
plt.show()

# 4. Modeling

## 4.1 Linear Regression (with manual AR1-MA1 dataset)

In [ ]:
# Features / target separation
features = df_res.drop(columns='comptage_horaire')
target = df_res['comptage_horaire']
test_ratio = 0.10

In [ ]:
# train / test separation avec ajout colonne
index_ratio = int(df_res.shape[0]*test_ratio//1)
X_train = features.iloc[:-index_ratio].copy()
X_test = features.iloc[-index_ratio:].copy()
y_train = target.iloc[:-index_ratio].copy()
y_test = target.iloc[-index_ratio:].copy()
# Création de la moyenne glissante (24 autour de t) et de la valeur t-1
X_train['comptage_horaire_lag1'] = y_train.shift(1)
X_train['comptage_horaire_MA24'] = y_train.rolling(window=24).mean()
mask_train = X_train.notna().all(axis=1)
X_train = X_train.loc[mask_train]
y_train = y_train.loc[mask_train]
X_test['comptage_horaire_lag1'] = y_test.shift(1)
X_test['comptage_horaire_MA24'] = y_test.rolling(window=24).mean()
mask_train = X_test.notna().all(axis=1)
X_test = X_test.loc[mask_train]
y_test = y_test.loc[mask_train]

display(X_train.shape, y_train.shape)
display(X_test.shape, y_test.shape)

In [ ]:
# scaling
# Preprocessing des variables explicatives d'entrainement et de test (scaler)
s_scaler = StandardScaler().fit(X_train)
# Recuperation des propriétés du dataframe perdues (nparray) avec le scaler
X_train = pd.DataFrame(s_scaler.fit_transform(X_train), index=X_train.index)
X_test = pd.DataFrame(s_scaler.transform(X_test), index=X_test.index)

In [ ]:
clf_lr = LinearRegression()
clf_lr.fit(X_train, y_train)

In [ ]:
y_test_pred = clf_lr.predict(X_test)
rmse = root_mean_squared_error(y_test, y_test_pred)
print(f'Erreur quadratique moyenne (RMSE) : {rmse}')

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(y_test.index, y_test, label='Valeurs réelles')  # type: ignore
plt.plot(y_test.index, y_test_pred, label='Prédictions', linestyle='--')  # type: ignore
plt.title('Prédictions de la régression linéaire')
plt.xlabel('Date')
plt.xlim([pd.to_datetime('2025-04-01'), pd.to_datetime('2025-04-16')])
plt.ylabel('Comptage Horaire')
plt.legend()
plt.show()

## 4.2 ARIMA / SARIMA

#### Features / target separation

In [ ]:
# Features / target separation
features = df_gpd.drop(columns=['comptage_horaire'])
target = df_gpd['comptage_horaire']
test_ratio = 0.10

#### Train / test separation (temporal context)

In [ ]:
# train / test separation 
index_ratio = int(df_gpd.shape[0]*test_ratio//1)
X_train = features.iloc[:-index_ratio].copy()
X_test = features.iloc[-index_ratio:].copy()
y_train = target.iloc[:-index_ratio].copy()
y_test = target.iloc[-index_ratio:].copy()

display(X_train.shape, y_train.shape)
display(X_test.shape, y_test.shape)

#### Standardisation

In [ ]:
# standardisation des échelles
# Preprocessing des variables explicatives d'entrainement et de test (scaler)
s_scaler = StandardScaler().fit(X_train)
# Recuperation des propriétés du dataframe perdues (nparray) avec le scaler
X_train = pd.DataFrame(s_scaler.fit_transform(X_train), index=X_train.index)
X_test = pd.DataFrame(s_scaler.transform(X_test), index=X_test.index)

#### Analyse de la saisonnalité par ACF/PCF

In [ ]:
# paramètres de saisonnalité
max_lag1 = s1*4
seasonal_lags1 = [k * s1 for k in range(1, (max_lag1 // s1) + 1)]
max_lag2 = s2*4
seasonal_lags2 = [k * s2 for k in range(1, (max_lag2 // s2) + 1)]

def ajouter_lignes_saisonnieres(ax, seasonal_lags):
    for lag in seasonal_lags:
        ax.axvline(x=lag, color='red', linestyle='--', alpha=0.6)

In [ ]:
# pré visualisation de :
# - l'autocorrelation simple (ACF - basé sur les X précédentes observations) 
# - l'autocorrelation partielle (PACF basée sur les moyennes mobiles d'amplitude X)
# Ces éléments permettent de pré-identifier les hypoerparamètres de SARIMA / ARIMA
# - d/D selon si les ACF et PACF tendent vers 0 ou stoppent net (ARMA si les deux tendent vers 0)
# - p/q grâce au 1er lag ACF/PACF entrant dans la zone statistiquement peu probable
# - P/Q grâce au 1er pic de lag cyclique (à motif saisonnier) ACF/PACF entrant dans la première zone statistiquement peu probable
fig2, (
    (ax21, ax22), 
    (ax23, ax24), 
    (ax25, ax26)
) = plt.subplots(3, 2, figsize=(20,20))

plot_acf(y, lags = max_lag1, ax=ax21)
ax21.set_title(f'ACF sans différenciation')
plot_pacf(y, lags = max_lag1, ax=ax22)
ax22.set_title(f'PACF sans différenciation')
plot_acf(y_s1, lags = max_lag1, ax=ax23)
ajouter_lignes_saisonnieres(ax23, seasonal_lags1)
ax23.set_title(f'ACF avec 1ère différenciation {s1} lags (heures)')
plot_pacf(y_s1, lags = max_lag1, ax=ax24)
ajouter_lignes_saisonnieres(ax24, seasonal_lags1)
ax24.set_title(f'PACF avec 1ère différenciation {s1} lags (heures)')
plot_acf(y_s2, lags = max_lag2, ax=ax25)
ajouter_lignes_saisonnieres(ax25, seasonal_lags2)
ax25.set_title(f'ACF avec 2ème différenciation {s2} lags (heures)')
plot_pacf(y_s2, lags = max_lag2, ax=ax26)
ajouter_lignes_saisonnieres(ax26, seasonal_lags2)
ax26.set_title(f'PACF avec 2ème différenciation {s2} lags (heures)')
plt.show()

>NB: Pour une double saisonalité passer a des modèles plus robustes comme TBATS ou BATS ou alors Fourier terms + ARIMA
>
>NB2: les différenciation exploratoires initiales de saisonalité sont la pour donner un indice il faut appliquer les modèles sur les données non différenciées néanmoins

In [ ]:
clf_sarima = SARIMAX(y_train, order=(1,1,1), seasonal_order=(1,1,1,24)) 
result_sarima = clf_sarima.fit()
print(result_sarima.summary())  # type: ignore

In [ ]:
n_test = len(y_test)
assert X_test.shape[0] == n_test
y_test_pred = result_sarima.get_forecast(  # type: ignore    
    steps=n_test, 
).summary_frame()
display(y_test_pred)
fig_sarima, ax_sarima = plt.subplots(figsize = (15,5))
plt.plot(y_test.index.to_timestamp(), y_test, label='Valeurs réelles')  # type: ignore
plt.plot(y_test_pred["mean"].index.to_timestamp(), y_test_pred["mean"], label='Prédictions', linestyle='--')
# ax_sarima.fill_between(y_test_pred.index, y_test_pred['mean_ci_lower'], y_test_pred['mean_ci_upper'], color='orange', alpha=0.1);
# # Ligne de démarcation du début des forecasts
# plt.axvline(x= datetime.date(2025,4,15), color='red')
plt.title('Prédictions de SARIMA')
plt.xlabel('Date')
plt.xlim([pd.to_datetime('2025-04-01'), pd.to_datetime('2025-04-16')])
plt.ylabel('Comptage Horaire')
plt.legend()
plt.show()

In [ ]:
# # Coute environ 12 minutes de CPU
# clf_sarimax = SARIMAX(y_train[-2000:], X_train[-2000:], order=(3,1,4),seasonal_order=(1,1,1,24))
# result_sarimax=clf_sarimax.fit()
# print(result_sarimax.summary())

In [ ]:
# # Sauvegarde avec pickle
# with open("sarimax_model.pkl", "wb") as f:
#     pickle.dump(result_sarimax, f)

In [ ]:
# # Chargement avec pickle depuis le modèle enregistré précédemment (si pas réentrainé)
# with open("sarimax_model.pkl", "rb") as f:
#     result_sarimax = pickle.load(f)

In [ ]:
# n_test = len(y_test)
# assert X_test.shape[0] == n_test
# y_test_pred = result_sarimax.get_forecast(    
#     steps=n_test, 
#     exog=X_test
# ).summary_frame()
# display(y_test_pred)
# fig_sarimax, ax_sarimax = plt.subplots(figsize = (15,5))
# plt.plot(y_test.index.to_timestamp(), y_test, label='Valeurs réelles')  # type: ignore
# plt.plot(y_test_pred['mean'].index.to_timestamp(), y_test_pred['mean'], label='Prédictions', linestyle='--')
# ax_sarimax.fill_between(y_test_pred.index, y_test_pred['mean_ci_lower'], y_test_pred['mean_ci_upper'], color='orange', alpha=0.1)
# # # Ligne de démarcation du début des forecasts
# # plt.axvline(x= datetime.date(2025,4,15), color='red')
# plt.title('Prédictions de SARIMA avec variables exogènes (X)')
# plt.xlabel('Date')
# plt.xlim([pd.to_datetime('2025-04-01'), pd.to_datetime('2025-04-16')])
# plt.ylabel('Comptage Horaire')
# plt.legend()
# plt.show()